In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('1128_output.csv')

In [3]:
# 找出TAICCA_ID中沒有「/」且min_publish_date為空的資料
mask = (~df['TAICCA_ID'].str.contains('/', na=False)) & (df['min_publish_date'].isna())

# 對這些資料，從各平台的發布日期中找出有值的日期來填補
for idx in df[mask].index:
    # 收集各平台的發布日期
    dates = []
    for col in ['bookscom_publish_date', 'eslite_publish_date', 'kingstone_publish_date', 'sanmin_publish_date']:
        if pd.notna(df.loc[idx, col]) and str(df.loc[idx, col]).strip():
            dates.append(df.loc[idx, col])
    
    # 如果有找到日期，就填入min_publish_date和max_publish_date
    if dates:
        # 直接使用第一個找到的日期，不進行比較
        df.loc[idx, 'min_publish_date'] = dates[0]
        df.loc[idx, 'max_publish_date'] = dates[0]

print(f"已處理 {mask.sum()} 筆資料的發布日期")


已處理 5966 筆資料的發布日期


In [7]:
# 統一min_publish_date和max_publish_date的格式為YYYY-MM-DD
from datetime import datetime
import re

def standardize_date_format(date_str):
    """將各種日期格式統一為YYYY-MM-DD格式"""
    if pd.isna(date_str) or not str(date_str).strip():
        return date_str
    
    date_str = str(date_str).strip()
    
    # 如果已經是YYYY-MM-DD格式，直接返回
    if re.match(r'^\d{4}-\d{2}-\d{2}$', date_str):
        return date_str
    
    # 處理各種可能的日期格式
    try:
        # 嘗試解析M/D/YY格式 (如 7/8/25)
        if re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
            month, day, year = date_str.split('/')
            # 假設YY格式的年份都是20XX年
            year = f"20{year}"
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        
        # 嘗試解析YYYY-M-D格式
        if re.match(r'^\d{4}-\d{1,2}-\d{1,2}$', date_str):
            parts = date_str.split('-')
            return f"{parts[0]}-{parts[1].zfill(2)}-{parts[2].zfill(2)}"
        
        # 嘗試其他常見格式
        for fmt in ['%Y-%m-%d', '%Y/%m/%d', '%m/%d/%Y', '%d/%m/%Y']:
            try:
                parsed_date = datetime.strptime(date_str, fmt)
                return parsed_date.strftime('%Y-%m-%d')
            except ValueError:
                continue
                
    except Exception as e:
        print(f"無法解析日期格式: {date_str}")
        return date_str
    
    return date_str

# 處理min_publish_date和max_publish_date欄位
for col in ['min_publish_date', 'max_publish_date', 'bookscom_publish_date', 'eslite_publish_date', 'kingstone_publish_date', 'sanmin_publish_date']:
    if col in df.columns:
        df[col] = df[col].apply(standardize_date_format)

print("已統一日期格式為YYYY-MM-DD")

已統一日期格式為YYYY-MM-DD


In [9]:
# 以title進行A-Z排序
# 首先檢查是否有title相關的欄位
title_columns = [col for col in df.columns if 'title' in col.lower()]
print(f"找到的title相關欄位: {title_columns}")

# 使用主要的title欄位進行排序，如果沒有則使用第一個找到的title欄位
if 'title' in df.columns:
    sort_column = 'title'
elif 'bookscom_title' in df.columns:
    sort_column = 'bookscom_title'
elif title_columns:
    sort_column = title_columns[0]
else:
    print("警告: 找不到title欄位，將使用第一個欄位進行排序")
    sort_column = df.columns[0]

print(f"使用 '{sort_column}' 欄位進行A-Z排序")

# 進行排序，將空值放在最後
df = df.sort_values(by=sort_column, ascending=True, na_position='last')

# 重置索引
df = df.reset_index(drop=True)

print(f"排序完成，共 {len(df)} 筆資料")


找到的title相關欄位: ['title', 'bookscom_title', 'eslite_title', 'kingstone_title', 'sanmin_title', 'processed_title', 'bookscom_processed_title', 'eslite_processed_title', 'kingstone_processed_title', 'sanmin_processed_title', 'original_title', 'bookscom_original_title', 'eslite_original_title', 'kingstone_original_title', 'sanmin_original_title']
使用 'title' 欄位進行A-Z排序
排序完成，共 10260 筆資料


In [10]:
df.to_csv('1201_output.csv', index=False, encoding='utf-8')

In [11]:
df_source = pd.read_csv('input_data/paper_source_1124.csv')

/var/folders/_2/w3bch_7n5c9_zvnqztg1b6tc0000gn/T/ipykernel_33830/1774769435.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,19,20,21,22,24,25,26,27,29,30,31,32,34,35,36,37,39,40,41,42,43,45,46,47,48,49,50,51,52,54,55,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df_source = pd.read_csv('input_data/paper_source_1124.csv')


In [12]:
# 找出未合併資料（TAICCA_ID 中沒有「/」）
unmerged_mask = ~df_source['TAICCA_ID'].str.contains('/', na=False)

# 定義需要填補的欄位及其對應的各平台欄位
fill_mapping = {
    'title': ['bookscom_title', 'eslite_title', 'kingstone_title', 'sanmin_title'],
    'isbn': ['bookscom_isbn', 'eslite_isbn', 'kingstone_isbn', 'sanmin_isbn'],
    'processed_title': ['bookscom_processed_title', 'eslite_processed_title', 'kingstone_processed_title', 'sanmin_processed_title'],
    'original_title': ['bookscom_original_title', 'eslite_original_title', 'kingstone_original_title', 'sanmin_original_title'],
    'author': ['bookscom_author', 'eslite_author', 'kingstone_author', 'sanmin_author'],
    'translator': ['bookscom_translator', 'eslite_translator', 'kingstone_translator', 'sanmin_translator'],
    'publisher': ['bookscom_publisher', 'eslite_publisher', 'kingstone_publisher', 'sanmin_publisher']
}

# 對每個未合併的資料進行填補
fill_count = 0
for idx in df_source[unmerged_mask].index:
    has_filled = False
    
    # 處理一般欄位
    for target_col, source_cols in fill_mapping.items():
        # 如果目標欄位是空的
        if pd.isna(df_source.loc[idx, target_col]) or not str(df_source.loc[idx, target_col]).strip():
            # 從各平台欄位中找第一個有值的
            for source_col in source_cols:
                if pd.notna(df_source.loc[idx, source_col]) and str(df_source.loc[idx, source_col]).strip():
                    df_source.loc[idx, target_col] = df_source.loc[idx, source_col]
                    has_filled = True
                    break
    
    if has_filled:
        fill_count += 1

print(f"已填補 {fill_count} 筆未合併資料的基本欄位（title, isbn, processed_title, original_title, author, translator, publisher）")


已填補 17216 筆未合併資料的基本欄位（title, isbn, processed_title, original_title, author, translator, publisher）


In [13]:
# 處理 min_publish_date 和 max_publish_date
date_fill_count = 0
for idx in df_source[unmerged_mask].index:
    # 如果 min_publish_date 或 max_publish_date 是空的
    if pd.isna(df_source.loc[idx, 'min_publish_date']) or pd.isna(df_source.loc[idx, 'max_publish_date']):
        # 收集各平台的發布日期
        dates = []
        for col in ['bookscom_publish_date', 'eslite_publish_date', 'kingstone_publish_date', 'sanmin_publish_date']:
            if pd.notna(df_source.loc[idx, col]) and str(df_source.loc[idx, col]).strip():
                dates.append(df_source.loc[idx, col])
        
        # 如果有找到日期，就填入（使用第一個找到的日期）
        if dates:
            if pd.isna(df_source.loc[idx, 'min_publish_date']) or not str(df_source.loc[idx, 'min_publish_date']).strip():
                df_source.loc[idx, 'min_publish_date'] = dates[0]
            if pd.isna(df_source.loc[idx, 'max_publish_date']) or not str(df_source.loc[idx, 'max_publish_date']).strip():
                df_source.loc[idx, 'max_publish_date'] = dates[0]
            date_fill_count += 1

print(f"已填補 {date_fill_count} 筆未合併資料的發布日期（min_publish_date, max_publish_date）")


已填補 17211 筆未合併資料的發布日期（min_publish_date, max_publish_date）


In [14]:
# 處理 price（取各平台價格的最大值）
price_fill_count = 0
for idx in df_source[unmerged_mask].index:
    # 如果 price 是空的
    if pd.isna(df_source.loc[idx, 'price']) or str(df_source.loc[idx, 'price']).strip() == '':
        # 收集各平台的價格
        prices = []
        for col in ['bookscom_original_price', 'eslite_original_price', 'kingstone_original_price', 'sanmin_original_price']:
            if pd.notna(df_source.loc[idx, col]):
                try:
                    price = float(df_source.loc[idx, col])
                    if price > 0:  # 只考慮大於 0 的價格
                        prices.append(price)
                except:
                    pass
        
        # 如果有找到價格，取最大值填入
        if prices:
            df_source.loc[idx, 'price'] = max(prices)
            price_fill_count += 1

print(f"已填補 {price_fill_count} 筆未合併資料的價格（price，取最大值）")


已填補 17216 筆未合併資料的價格（price，取最大值）


In [15]:
# 儲存處理後的檔案
df_source.to_csv('input_data/paper_source_1201.csv', index=False, encoding='utf-8')
print("已儲存至 input_data/paper_source_1201.csv")


已儲存至 input_data/paper_source_1201.csv
